In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu May 20 23:36:05 2021

@author: wangrenrui
"""


import matplotlib.pyplot as plt
import numpy as np
import cv2 
import math
import imutils
from skimage import morphology

def grade (pho):
    
    image=cv2.GaussianBlur(pho,(3,3),0,0)
    sobelx=cv2.Sobel(image,cv2.CV_64F,1,0,ksize=3)
    sobelxx=sobelx**2
    sobely=cv2.Sobel(image,cv2.CV_64F,0,1,ksize=3)
    sobelyy=sobely**2
    sobelxy=(sobelxx+sobelyy)**0.5
    t=sobely/(sobelx+0.00000000000000001)
    yy=np.arctan(t)*180/math.pi
    yy=np.abs(yy)
    return sobelxy,yy
    
def selcetp (pho,angel,threshold,up,low):
    h=pho.shape[0]
    w=pho.shape[1]
    zero0=np.zeros((h,w))
    for i in range(0,h,1):
        for j in range(0,w,1):
            if pho[i,j]>=threshold and angel[i,j]<up+low and angel[i,j]>=up-low:
                zero0[i,j]=1
    return zero0

def link (zero1,n0):
    a=zero1.shape[0]
    b=zero1.shape[1]
    ident=0
    m=0
    n=0
    d=0
    z0=np.zeros((a,b))
    recal1=np.zeros((a,b))
    for i in range(0,a,1):
        ident=0
        for j in range(1,b-1,1):
            if zero1[i,j-1] != zero1[i,j+1]:
                ident=ident+1
                z0[i,j]=1
                recal1[i,ident]=j
    #进行连接
    for i in range(0,a,1):
        for j in range(1,b-1,1):
            m=recal1[i,j]
            n=recal1[i,j+1]
            d=n-m
            if d>0 and d<n0:
                z0[i,int(m):int(n)]=1;
    return z0





                 

pho=cv2.imread("/Users/wangrenrui/Desktop/OpenCv/实验三 2021\
/Fig1027(a)(van_original).tif",cv2.IMREAD_UNCHANGED)
height=pho.shape[0]
wide=pho.shape[1]
zero0=np.zeros((height,wide))
zero1=np.zeros((height,wide))
(sobelxy,yy)=grade(pho)
cv2.imwrite("/Users/wangrenrui/Desktop/OpenCv/课程实验图片/exp3/pho_sobelxy.jpg",sobelxy)
maxone=np.max(sobelxy)
low=20
high=90
rotated = imutils.rotate(pho, 90) #逆时针旋转
(sobelxy2,yy2)=grade(rotated)
zero0=selcetp (sobelxy,yy,maxone*0.4,high,low)
zero1=selcetp (sobelxy2,yy2,maxone*0.4,high,low)
zero2=imutils.rotate(zero1, -90)
tran1=cv2.bitwise_or(link(zero0,40),zero0)
tran2=cv2.bitwise_or(link(zero1,40),zero1)
tran2=imutils.rotate(tran2, -90)
tranor=cv2.bitwise_or(tran1,tran2)
tranor[tranor==255] = 1
skeleton0 = morphology.skeletonize(tranor)
skeleton = skeleton0.astype(np.uint8)*255
kernel=np.ones((3,1),np.uint8)
erosion=cv2.erode(tranor,kernel)
cv2.imwrite("/Users/wangrenrui/Desktop/OpenCv/课程实验图片/exp3/pho_zerosion.png",erosion*255)
    
cv2.imwrite("/Users/wangrenrui/Desktop/OpenCv/课程实验图片/exp3/pho_zero0.png",zero0*255)
cv2.imwrite("/Users/wangrenrui/Desktop/OpenCv/课程实验图片/exp3/pho_zero1.png",zero2*255)
cv2.imwrite("/Users/wangrenrui/Desktop/OpenCv/课程实验图片/exp3/pho_tran1.png",tran1*255)
cv2.imwrite("/Users/wangrenrui/Desktop/OpenCv/课程实验图片/exp3/pho_tran2.png",tran2*255)
cv2.imwrite("/Users/wangrenrui/Desktop/OpenCv/课程实验图片/exp3/pho_tranor.png",tranor*255)
cv2.imwrite("/Users/wangrenrui/Desktop/OpenCv/课程实验图片/exp3/skeleton.png",skeleton)